In [1]:
# Notebook that will be used to compute the metrics used to select to best model.
# Input : model, first timestep and number of timestep to predict
# Output : csv file with ts, time, metrics, pictures of wavefields to save and receivers data

In [1]:
import sys

sys.path.append('C:/Users/nilso/Documents/EPFL/PDM/PDM_PINN/SciANN/DNN_TEST/sys/')

from loss import *
from unet import UNet
from encoder import AutoEncoder
from multi_scale import MultiScale
from dataloader import *
from BaseModel import BaseModel
from recursive_predictions import *

import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1 import make_axes_locatable

from kornia.filters import spatial_gradient

import h5py

import torch.optim as optim
import time
import json

In [2]:
class Model(BaseModel):
    def __init__(self, net, opt=None, sched=None, logger=None, print_progress=True, device='cuda:0'):
        super().__init__(net, opt, sched, logger, print_progress, device)

    def forward_loss(self, data):
        return

In [3]:
# # # Data
training_data = dataset('Training_Data/Moseley_Homogeneous/','Training_Data/Moseley_Homogeneous.csv','Event0000')

model_name = 'AR_PINN_MSE_E500'
model_type = 'multiscale'

#net = UNet(in_channels=4,out_channels=1)
#net = AutoEncoder(in_channels=4,out_channels=1)
net = MultiScale(in_channels=4)

model = Model(net, device='cuda:0')
model.load(import_path=f'Training/{model_type}/results/{model_name}.pt',map_location='cuda:0')

predictions = RecursivePredictions(net=model.net,dataset=training_data)

# Load receivers data to obtain the coordinates
path = f"E:/PDM_SALVUS/NoCrack/Moseley_Homogeneous/Event0000/meta.json"
f = open(path,)
data = json.load(f)

# # # Parameters for the comparison
initial_timestep = 0
number_of_timestep_to_predict = 151 # 151 max
time_timestep = 0.002
timestep_to_plot = [0,25,50,100,150]

x_locations = pd.DataFrame.from_dict(data['forward_run_input']['output']['point_data']['receiver'])['location'].apply(pd.Series).astype(int)[0].values
y_locations = pd.DataFrame.from_dict(data['forward_run_input']['output']['point_data']['receiver'])['location'].apply(pd.Series).astype(int)[1].values

x_locations = np.where(x_locations == 1500, 1495, x_locations)
y_locations = np.where(y_locations == 1500, 1495, y_locations)

dh = 5

# # # Names
results_name = f"{model_type}_{model_name}.csv"
errors_name = f"model_comparison/{model_type}_errors_{model_name}.npy"
preds_name = f"model_comparison/{model_type}_preds_{model_name}.npy"
trues_name = f"model_comparison/{model_type}_trues_{model_name}.npy"

save_path = 'model_comparison/' + results_name 

# # # Predictions
preds, trues, errors, dict_metrics = predictions.predict_all(initial_timestep=initial_timestep,number_of_timestep_to_predict=number_of_timestep_to_predict,time_timestep=time_timestep,timestep_to_plot=timestep_to_plot,x_locations=x_locations,y_locations=y_locations,dh=dh,model=model_type)

# # # Save
pd.DataFrame.from_dict(dict_metrics).to_csv(save_path)

for filename, object in zip([errors_name,trues_name,preds_name],[errors,trues,preds]):
    np.save(filename, object) 

Saving wavefield 0

Total time for the timestep 0 in seconds 0.98

Total time for the timestep 1 in seconds 0.11

Total time for the timestep 2 in seconds 0.12

Total time for the timestep 3 in seconds 0.11

Total time for the timestep 4 in seconds 0.11

Total time for the timestep 5 in seconds 0.11

Total time for the timestep 6 in seconds 0.12

Total time for the timestep 7 in seconds 0.11

Total time for the timestep 8 in seconds 0.11

Total time for the timestep 9 in seconds 0.11

Total time for the timestep 10 in seconds 0.11

Total time for the timestep 11 in seconds 0.11

Total time for the timestep 12 in seconds 0.12

Total time for the timestep 13 in seconds 0.11

Total time for the timestep 14 in seconds 0.12

Total time for the timestep 15 in seconds 0.11

Total time for the timestep 16 in seconds 0.11

Total time for the timestep 17 in seconds 0.12

Total time for the timestep 18 in seconds 0.12

Total time for the timestep 19 in seconds 0.11

Total time for the timestep 20

KeyboardInterrupt: 